# MCP Server Deployment - Interactive Exploration

**Purpose:** Interactive notebook for exploring OAuth 2.1 + PKCE flow and MCP protocol testing.

**Not for Production:** This notebook is a learning tool. For actual deployment, use the Python template file and deployment guide.

---

## Prerequisites

- MCP server running locally (follow `DEPLOYMENT_GUIDE_PYTHON.md`)
- Server accessible at `http://localhost:8080`
- DCR_AUTH_TOKEN from your `.env` file

---

## Setup: Install Required Packages

Run this cell to install dependencies for the notebook:

In [ ]:
!pip install requests python-dotenv -q

## Configuration

Update these values to match your environment:

In [ ]:
import os
from dotenv import load_dotenv

# Load from .env file (optional)
load_dotenv()

# Server configuration
SERVER_URL = os.getenv('SERVER_URL', 'http://localhost:8080')
DCR_AUTH_TOKEN = os.getenv('DCR_AUTH_TOKEN', 'your-dcr-token-here')
REDIRECT_URI = 'http://localhost:3000/callback'

print(f"✅ Server URL: {SERVER_URL}")
print(f"✅ DCR Token: {DCR_AUTH_TOKEN[:10]}...")

---

## Phase 1: Health Check

Verify the MCP server is running and healthy:

In [ ]:
import requests
import json

def pretty_print(data):
    """Pretty print JSON data"""
    print(json.dumps(data, indent=2))

# Health check
response = requests.get(f"{SERVER_URL}/health")
print(f"Status Code: {response.status_code}")
pretty_print(response.json())

if response.status_code == 200:
    print("\n✅ Server is healthy and ready!")
else:
    print("\n❌ Server health check failed!")

---

## Phase 2: OAuth Metadata Discovery

Fetch OAuth server metadata (RFC 8414):

In [ ]:
# OAuth Authorization Server Metadata
response = requests.get(f"{SERVER_URL}/.well-known/oauth-authorization-server")
metadata = response.json()

print("OAuth Server Metadata:")
pretty_print(metadata)

print("\n📋 Available Endpoints:")
print(f"  Authorization: {metadata['authorization_endpoint']}")
print(f"  Token: {metadata['token_endpoint']}")
print(f"  Revocation: {metadata['revocation_endpoint']}")
print(f"  Registration: {metadata['registration_endpoint']}")

---

## Phase 3: Dynamic Client Registration (DCR)

Register a new OAuth client with the MCP server:

In [ ]:
# Register client via DCR
dcr_response = requests.post(
    f"{SERVER_URL}/oauth/register",
    headers={
        'Authorization': f'Bearer {DCR_AUTH_TOKEN}',
        'Content-Type': 'application/json'
    },
    json={
        'client_name': 'Jupyter Notebook Client',
        'redirect_uris': [REDIRECT_URI]
    }
)

if dcr_response.status_code == 201:
    client_data = dcr_response.json()
    CLIENT_ID = client_data['client_id']
    CLIENT_SECRET = client_data['client_secret']
    
    print("✅ Client registered successfully!\n")
    print(f"Client ID: {CLIENT_ID}")
    print(f"Client Secret: {CLIENT_SECRET}")
    print(f"\n⚠️  IMPORTANT: Save these credentials! Client secret is only shown once.")
else:
    print(f"❌ DCR failed: {dcr_response.status_code}")
    print(dcr_response.text)

---

## Phase 4: Generate PKCE Parameters

Generate code verifier and challenge for PKCE (Proof Key for Code Exchange):

In [ ]:
import secrets
import hashlib
import base64

def generate_pkce_pair():
    """Generate PKCE code_verifier and code_challenge"""
    # Generate code_verifier (43-128 characters)
    code_verifier = base64.urlsafe_b64encode(secrets.token_bytes(32)).decode('utf-8').rstrip('=')
    
    # Generate code_challenge (SHA256 hash of verifier)
    challenge_bytes = hashlib.sha256(code_verifier.encode('utf-8')).digest()
    code_challenge = base64.urlsafe_b64encode(challenge_bytes).decode('utf-8').rstrip('=')
    
    return code_verifier, code_challenge

# Generate PKCE parameters
CODE_VERIFIER, CODE_CHALLENGE = generate_pkce_pair()

print("PKCE Parameters Generated:")
print(f"\nCode Verifier: {CODE_VERIFIER}")
print(f"Code Challenge: {CODE_CHALLENGE}")
print(f"Challenge Method: S256")

print("\n✅ PKCE parameters ready for authorization request")

---

## Phase 5: Authorization Request

**⚠️ MANUAL STEP REQUIRED:**

Open the authorization URL below in your browser. The server will redirect you to the callback URL with an authorization code.

Copy the `code` parameter from the redirect URL and paste it in the next cell.

In [ ]:
from urllib.parse import urlencode

# Build authorization URL
auth_params = {
    'client_id': CLIENT_ID,
    'redirect_uri': REDIRECT_URI,
    'response_type': 'code',
    'code_challenge': CODE_CHALLENGE,
    'code_challenge_method': 'S256',
    'state': 'notebook_test',
    'scope': 'openid profile'
}

authorization_url = f"{SERVER_URL}/oauth/authorize?{urlencode(auth_params)}"

print("🔗 Authorization URL:")
print(f"\n{authorization_url}")
print("\n📝 Instructions:")
print("1. Click the URL above (or copy-paste into browser)")
print("2. Server will redirect to callback URL with authorization code")
print("3. Copy the 'code' parameter from the URL")
print("4. Paste it in the next cell")

**Paste the authorization code here:**

In [ ]:
# Paste authorization code from redirect URL
AUTHORIZATION_CODE = "paste-authorization-code-here"

print(f"✅ Authorization code received: {AUTHORIZATION_CODE[:20]}...")

---

## Phase 6: Token Exchange

Exchange the authorization code for access and refresh tokens:

In [ ]:
# Exchange authorization code for tokens
token_response = requests.post(
    f"{SERVER_URL}/oauth/token",
    data={
        'grant_type': 'authorization_code',
        'code': AUTHORIZATION_CODE,
        'code_verifier': CODE_VERIFIER,
        'redirect_uri': REDIRECT_URI,
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET
    },
    headers={'Content-Type': 'application/x-www-form-urlencoded'}
)

if token_response.status_code == 200:
    tokens = token_response.json()
    ACCESS_TOKEN = tokens['access_token']
    REFRESH_TOKEN = tokens['refresh_token']
    EXPIRES_IN = tokens['expires_in']
    
    print("✅ Tokens received successfully!\n")
    print(f"Access Token: {ACCESS_TOKEN[:30]}...")
    print(f"Refresh Token: {REFRESH_TOKEN[:30]}...")
    print(f"Expires In: {EXPIRES_IN} seconds ({EXPIRES_IN/60} minutes)")
    print(f"Token Type: {tokens['token_type']}")
else:
    print(f"❌ Token exchange failed: {token_response.status_code}")
    print(token_response.text)

---

## Phase 7: Test MCP Protocol

Now that we have an access token, let's test the MCP endpoints:

### 7.1 Initialize Handshake (No Auth Required)

In [ ]:
# MCP Initialize
init_response = requests.post(
    f"{SERVER_URL}/mcp",
    json={
        'jsonrpc': '2.0',
        'method': 'initialize',
        'params': {
            'protocolVersion': '2025-06-18',
            'capabilities': {}
        },
        'id': 1
    }
)

print("MCP Initialize Response:")
pretty_print(init_response.json())

if init_response.status_code == 200:
    print("\n✅ MCP handshake successful!")

### 7.2 List Available Tools (Requires Auth)

In [ ]:
# MCP Tools List
tools_response = requests.post(
    f"{SERVER_URL}/mcp",
    headers={'Authorization': f'Bearer {ACCESS_TOKEN}'},
    json={
        'jsonrpc': '2.0',
        'method': 'tools/list',
        'params': {},
        'id': 2
    }
)

print("Available Tools:")
tools = tools_response.json()['result']['tools']
for tool in tools:
    print(f"\n📦 {tool['name']}")
    print(f"   {tool['description']}")
    print(f"   Required params: {tool['inputSchema'].get('required', [])}")

print(f"\n✅ Found {len(tools)} tools")

### 7.3 Execute Tool (Requires Auth)

In [ ]:
# MCP Tool Call - Echo
tool_response = requests.post(
    f"{SERVER_URL}/mcp",
    headers={'Authorization': f'Bearer {ACCESS_TOKEN}'},
    json={
        'jsonrpc': '2.0',
        'method': 'tools/call',
        'params': {
            'name': 'echo',
            'arguments': {
                'message': 'Hello from Jupyter Notebook!'
            }
        },
        'id': 3
    }
)

print("Tool Execution Result:")
result = tool_response.json()['result']
print(result['content'][0]['text'])

if tool_response.status_code == 200:
    print("\n✅ Tool execution successful!")

### 7.4 Calculate Tool Example

In [ ]:
# MCP Tool Call - Calculate
calc_response = requests.post(
    f"{SERVER_URL}/mcp",
    headers={'Authorization': f'Bearer {ACCESS_TOKEN}'},
    json={
        'jsonrpc': '2.0',
        'method': 'tools/call',
        'params': {
            'name': 'calculate',
            'arguments': {
                'operation': 'multiply',
                'a': 42,
                'b': 7
            }
        },
        'id': 4
    }
)

print("Calculation Result:")
result = calc_response.json()['result']
print(result['content'][0]['text'])

---

## Phase 8: Token Refresh

Use the refresh token to get new access and refresh tokens:

In [ ]:
# Refresh tokens
refresh_response = requests.post(
    f"{SERVER_URL}/oauth/token",
    data={
        'grant_type': 'refresh_token',
        'refresh_token': REFRESH_TOKEN,
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET
    },
    headers={'Content-Type': 'application/x-www-form-urlencoded'}
)

if refresh_response.status_code == 200:
    new_tokens = refresh_response.json()
    NEW_ACCESS_TOKEN = new_tokens['access_token']
    NEW_REFRESH_TOKEN = new_tokens['refresh_token']
    
    print("✅ Tokens refreshed successfully!\n")
    print(f"New Access Token: {NEW_ACCESS_TOKEN[:30]}...")
    print(f"New Refresh Token: {NEW_REFRESH_TOKEN[:30]}...")
    print(f"\n⚠️  Old refresh token is now invalid (token rotation)")
    
    # Update tokens for subsequent calls
    ACCESS_TOKEN = NEW_ACCESS_TOKEN
    REFRESH_TOKEN = NEW_REFRESH_TOKEN
else:
    print(f"❌ Token refresh failed: {refresh_response.status_code}")
    print(refresh_response.text)

---

## Phase 9: Token Revocation

Revoke the access token (invalidates it immediately):

In [ ]:
# Revoke token
revoke_response = requests.post(
    f"{SERVER_URL}/oauth/revoke",
    data={
        'token': ACCESS_TOKEN,
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET
    },
    headers={'Content-Type': 'application/x-www-form-urlencoded'}
)

if revoke_response.status_code == 200:
    print("✅ Token revoked successfully!")
    print("\nThe access token is now invalid and cannot be used.")
else:
    print(f"❌ Revocation failed: {revoke_response.status_code}")

### Verify Token is Revoked

In [ ]:
# Try to use revoked token
verify_response = requests.post(
    f"{SERVER_URL}/mcp",
    headers={'Authorization': f'Bearer {ACCESS_TOKEN}'},
    json={
        'jsonrpc': '2.0',
        'method': 'tools/list',
        'params': {},
        'id': 5
    }
)

if verify_response.status_code == 401:
    print("✅ Token revocation verified!")
    print("Server correctly rejected the revoked token.")
    print(f"\nError: {verify_response.json()['error']['message']}")
else:
    print(f"⚠️  Unexpected response: {verify_response.status_code}")

---

## Summary

### What We Accomplished:

✅ **Health Check** - Verified server is running  
✅ **OAuth Metadata** - Discovered available endpoints  
✅ **DCR** - Registered OAuth client dynamically  
✅ **PKCE** - Generated code verifier and challenge  
✅ **Authorization** - Obtained authorization code  
✅ **Token Exchange** - Got access and refresh tokens  
✅ **MCP Protocol** - Tested initialize, tools/list, tools/call  
✅ **Token Refresh** - Rotated tokens (old refresh token invalidated)  
✅ **Token Revocation** - Invalidated token and verified blacklist  

### OAuth 2.1 + PKCE Flow Validated:

```
1. Client Registration (DCR)
   ↓
2. Generate PKCE parameters
   ↓
3. Authorization request (with code_challenge)
   ↓
4. Authorization code issued
   ↓
5. Token exchange (with code_verifier)
   ↓
6. Access + Refresh tokens issued
   ↓
7. Use access token for MCP calls
   ↓
8. Refresh tokens (old refresh token revoked)
   ↓
9. Revoke token (blacklist verified)
```

### Next Steps:

- **For Production Deployment:** Follow `DEPLOYMENT_GUIDE_PYTHON.md`
- **For Custom Tools:** Modify template file to add your business logic
- **For ServiceNow Integration:** Share DCR_AUTH_TOKEN with ServiceNow team

---

## Appendix: Helper Functions

Reusable functions for MCP testing:

In [ ]:
def call_mcp_tool(tool_name: str, arguments: dict, access_token: str) -> dict:
    """Helper function to call MCP tools"""
    response = requests.post(
        f"{SERVER_URL}/mcp",
        headers={'Authorization': f'Bearer {access_token}'},
        json={
            'jsonrpc': '2.0',
            'method': 'tools/call',
            'params': {
                'name': tool_name,
                'arguments': arguments
            },
            'id': 999
        }
    )
    return response.json()

def decode_jwt(token: str) -> dict:
    """Decode JWT token (without verification)"""
    import base64
    # Split token and decode payload
    parts = token.split('.')
    payload = parts[1] + '=' * (4 - len(parts[1]) % 4)  # Add padding
    decoded = base64.urlsafe_b64decode(payload)
    return json.loads(decoded)

# Example: Decode access token to see claims
if 'ACCESS_TOKEN' in locals():
    claims = decode_jwt(ACCESS_TOKEN)
    print("Access Token Claims:")
    pretty_print(claims)